# Support Vector Machines

A *Support Vector Machine* (SVM) is a powerful and versatile Machine Learning model, capable of performing linear or nonlinear classification, regression, and even outlier detection. SVMs are particularly well suited for classification of complex small- or medium-sized datasets.

## Linear SVM Classification

Figure 5.1 good explanaiton

You can thjink of an SVM classifier as fitting the widest possible street (represented by the parallel dashed lines) between the class. This is called *larged margin classification*.

Notice that adding more training instances "off the street" will not affect the decision boundary at all: it is fully determined (or "supported") by the instances located on the edge of the street. These instances are called the *support vectors*.

SVMs are sensitive to the feature scales.

## Soft Margin Classification

If we strictly impose all instances must be off the street and on the right side, this is called *hard margin classification*.The two main issued are:
- It only work if the data is linearly separable.
- It is sensitive to outliers

The objective is to find a good balance between keeping, the street as large as possible and limiting the *margin violations* (i.e., instances that ends up in the middle of the street or even on the wrong side). This is called *soft margin classification*.

If your SVM model is overfitting, you can try regularizing it by reducing C.

In [5]:
import numpy as np
from sklearn import datasets
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC

iris = datasets.load_iris()
X = iris["data"][:, (2,3)]  #petal length, petal width
y = (iris["target"] == 2).astype(np.float64) #Iris virginica

svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("linear_svc", LinearSVC(C=1, loss="hinge")),
])

svm_clf.fit(X, y)

svm_clf.predict([[5.5, 1.7]])

array([1.])

Unlike Logistic Reegression classifiers, SVM classifiers do not output probabilities for each class.

Instead of using the LinearSVC class, we could use the SVC class with a linear kernel (SVC(kernel="linear", C=1)).

For better performance ypus hould set the dual hyperparameter to False, unless there are more features than training instances.

## Nonlinear SVM Classification

One approach to handling nonlinear datasetsis to add more features, such as polynomial features; in some cases this can result in a linearly separable dataset.

In [14]:
from sklearn.datasets import make_moons
from sklearn.preprocessing import PolynomialFeatures

X, y = make_moons(n_samples=100, noise=0.15)
polynomial_svm_clf = Pipeline([
    ("poly_features", PolynomialFeatures(degree=3)),
    ("scaler", StandardScaler()),
    ("svm_clf", LinearSVC(C=10, loss="hinge",max_iter=10000)) #Increase the iterations
])

polynomial_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('poly_features',
                 PolynomialFeatures(degree=3, include_bias=True,
                                    interaction_only=False, order='C')),
                ('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 LinearSVC(C=10, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='hinge', max_iter=10000, multi_class='ovr',
                           penalty='l2', random_state=None, tol=0.0001,
                           verbose=0))],
         verbose=False)

### Polynomial Kernel

Adding polynomial features is simple to implement and can work great with all sorts of Machine Learning algorithms. 

Fortunately, when using SVMs you can apply an almost miraculous mathematical technique called the *kernel trick*. The kernel trick makes it possible to get the same result as if you added many polynomial features, even with very high-degree polynomials, wthout having actually to add them.

In [15]:
from sklearn.svm import SVC

poly_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="poly", degree=3, coef0=1, C=5))
])

poly_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=5, break_ties=False, cache_size=200, class_weight=None,
                     coef0=1, decision_function_shape='ovr', degree=3,
                     gamma='scale', kernel='poly', max_iter=-1,
                     probability=False, random_state=None, shrinking=True,
                     tol=0.001, verbose=False))],
         verbose=False)

### Similarity Features

Another technique to tackle nonlinear problems is to add features computed using a *similarity function*, which measures how much each instance a particular *landmark*.

**Equation Gaussian RBF similarity function**

$\phi_{y}(x,l) = exp (-\gamma \lVert x - l \rVert^{2})$

This is a bell-shaped function varying from 0 (very far away from the landmark) to 1 (at the landmark).

The simplest approach on select the landmarks is create one at the location of each and every instance in the dataset. Doing that creates many dimensions and thus increases the chances that the transformed training set will be linearly eparable. The downside is that the number of features is going to be the same as number of instances.

### Gaussian RBF Kernel

In [16]:
rbf_kernel_svm_clf = Pipeline([
    ("scaler", StandardScaler()),
    ("svm_clf", SVC(kernel="rbf", gamma=5, C=0.001))
])

rbf_kernel_svm_clf.fit(X, y)

Pipeline(memory=None,
         steps=[('scaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('svm_clf',
                 SVC(C=0.001, break_ties=False, cache_size=200,
                     class_weight=None, coef0=0.0,
                     decision_function_shape='ovr', degree=3, gamma=5,
                     kernel='rbf', max_iter=-1, probability=False,
                     random_state=None, shrinking=True, tol=0.001,
                     verbose=False))],
         verbose=False)

Other kernels exist but are used more rarely. Some kernels are specialized for specific data structures. *String kernels* are sometimes used when classifying text documents or DNA sequences.

As a rule of thumb, you should always try the linear kernel first, especially if the training set is very large or if it has plenty of features. If the training set is not too large, ypu should also try the Gaussian RBF kernel; it works well in most cases.

## SVM Regression

SVM algorithm is versatile: not only does it support linear and nonlinear classification, but it also supports linear and nonlinear regression.

the objective is the opposite one; try to fit as many instances as possible one the street while limiting margin violations. The width of the street is controlled by a hyperparameter, $\epsilon$.

In [17]:
from sklearn.svm import LinearSVR

svm_reg = LinearSVR(epsilon=1.5)
svm_reg.fit(X, y)

LinearSVR(C=1.0, dual=True, epsilon=1.5, fit_intercept=True,
          intercept_scaling=1.0, loss='epsilon_insensitive', max_iter=1000,
          random_state=None, tol=0.0001, verbose=0)

To tackle nonlinear regression tasks, you can use a kernelized SVM model.

In [18]:
from sklearn.svm import SVR

svm_poly_reg = SVR(kernel="poly", degree=2, C=100, epsilon=0.1)
svm_poly_reg.fit(X, y)

SVR(C=100, cache_size=200, coef0=0.0, degree=2, epsilon=0.1, gamma='scale',
    kernel='poly', max_iter=-1, shrinking=True, tol=0.001, verbose=False)

The SVR class is the regression equivalent of the SVC class, and the LinearSVR class is the regression equivalent of the LinearSVC class.

SVMs can also be used for outlier detection.

## Under the Hood

### Decision Function and Predictions

The linear SVM classifier model predicts the class of a new instance x by simply computting the decision function.

**Equation Linear SVM classifier prediction**

$\hat{y} =  \left\{ \begin{array}{lcc}
             0 &   if  & w^{T}x + b < 0 \\
             \\ 1 &  if  & w^{T}x + b \geq 0
             \end{array}
   \right.$
   
- w is the feature wieght vector (previously called $\theta$)
- b is the bias term (previously called x_{0} = 1)

### Training Objective

To get the soft margin objective, we need to introduce a *slack variable* for each instance: measures how much the $i^{th}$ instance is allowed to violate the margin.